In [22]:
# import os
# api_key = os.getenv("OPENAI_API_KEY")

## Import input data

In [1]:
def read_file(file_path):
    with open(file_path) as f:
        return f.readlines()

# Read job_info.txt,profile_info.txt,letter_raw.txt files
job_info = read_file("job_info.txt")
profile_info = read_file("profile_info.txt")
letter_raw = read_file("letter_raw.txt")

## Call the OpenAI API

In [3]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "Je suis un assistant qui permet de remplir les lettres de motivation pour vos candidatures. Je fais le lien entre vos expériences, vos compétences et l'offre d'emploi."},
    
    {"role": "user", "content": f"Voici mes compétences et expériences : {profile_info}"},

    {"role": "system", "content": "Donnez moi l'offre d'emploi."},

    {"role": "user", "content": f"Voici le contenu de l'offre d'emploi : {job_info}"},

    {"role": "system", "content": "Donne moi le format type de lettre de motivation que je dois suivre."},

    {"role": "user", "content": f"Voici le format type que j'aimerais que tu suives : {letter_raw}"},
    
  ]
)

print(completion.choices[0].message.content)

Poste & Entreprise : Bouygues Telecom - Alternance Développeur IA/Test F/H
Objet : Candidature pour une alternance en Développeur IA/Test F/H chez Bouygues Telecom

Madame, Monsieur,

Actuellement en préparation d'un Master 2 spécialisé en Big Data & Machine Learning à l'Efrei, je suis activement à la recherche d'une entreprise pour m'accueillir un an à partir d’octobre 2024. Cette expérience sera l’occasion d’enrichir ma formation théorique par une expérience pratique. C'est avec beaucoup d'enthousiasme que je vous présente ma candidature pour une alternance chez Bouygues Telecom pour le poste de Développeur IA/Test F/H.

Votre entreprise est reconnue pour son engagement dans l'innovation technologique et l'excellence de ses services dans le domaine des télécommunications. Je suis particulièrement intéressé par votre approche d'intégration de l'intelligence artificielle pour améliorer et optimiser les processus, ce qui représente une opportunité inestimable de contribuer à des projets

In [5]:
result_raw = completion.choices[0].message.content

## Data Cleaning

In [10]:
# First extraction: "Poste & Entreprise : " to before "Objet"
start_keyword_pe = "Poste & Entreprise : "
end_keyword_pe = "\nObjet"
start_index_pe = result_raw.find(start_keyword_pe) + len(start_keyword_pe)
end_index_pe = result_raw.find(end_keyword_pe)

# Second extraction: "Objet" to "DESCLOUS"
start_keyword_objet = "Objet"
end_keyword_objet = "DESCLOUS"
start_index_objet = result_raw.find(start_keyword_objet)
end_index_objet = result_raw.find(end_keyword_objet) + len(end_keyword_objet)

# Extracting the desired contents
poste_entreprise = result_raw[start_index_pe:end_index_pe].strip()
letter = result_raw[start_index_objet:end_index_objet].strip()


print(poste_entreprise)
print(letter)

Bouygues Telecom - Alternance Développeur IA/Test F/H
Objet : Candidature pour une alternance en Développeur IA/Test F/H chez Bouygues Telecom

Madame, Monsieur,

Actuellement en préparation d'un Master 2 spécialisé en Big Data & Machine Learning à l'Efrei, je suis activement à la recherche d'une entreprise pour m'accueillir un an à partir d’octobre 2024. Cette expérience sera l’occasion d’enrichir ma formation théorique par une expérience pratique. C'est avec beaucoup d'enthousiasme que je vous présente ma candidature pour une alternance chez Bouygues Telecom pour le poste de Développeur IA/Test F/H.

Votre entreprise est reconnue pour son engagement dans l'innovation technologique et l'excellence de ses services dans le domaine des télécommunications. Je suis particulièrement intéressé par votre approche d'intégration de l'intelligence artificielle pour améliorer et optimiser les processus, ce qui représente une opportunité inestimable de contribuer à des projets d'avant-garde.

En t

In [ ]:
poste_entreprise = poste_entreprise.replace("/","-")

link_to_the_job = job_info[0]
export = link_to_the_job + "\n\n" + letter

## Export result

In [27]:
# Write the content to the file
file_path = f"CL - {poste_entreprise}.txt"
with open(file_path, "w") as file:
    file.write(export)